In [1]:
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\y63qiu\AppData\Local\Continuum\anaconda3\envs\yorkml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets("C:\\Users\\y63qiu\\", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:\Users\y63qiu\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:\Users\y63qiu\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting C:\Users\y63qiu\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting C:\Users\y63qiu\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official

In [12]:
mnist.train.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [16]:
maxarg = tf.argmax(mnist.train.labels,1)

with tf.Session() as sess:
    print(sess.run(maxarg))
    

[7 3 4 ... 5 6 8]


In [4]:
print("Training data size:", mnist.train.num_examples)

Training data size: 55000


In [5]:
print("Validating data size:", mnist.validation.num_examples)

Validating data size: 5000


In [7]:
print('Testing data size:', mnist.test.num_examples)

Testing data size: 10000


## Import所需以及设置神经网络基本参数

In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 配置神经网络相关的参数

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGU_RATE = 0.0001

BATCH_SIZE = 100
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

## 定义一些辅助函数

In [50]:
# 这个函数用于计算神经网络的向前传播

def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 如果没有提供滑动平均类时，直接使用参数当前取值
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        # 计算出变量的滑动平均值，再计算向前传播的结果
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [51]:
# 这个函数用于定义神经网络训练的过程

def train(minst):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 定义各层的 weight 和 biases
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    y = inference(x, None, weights1, biases1, weights2, biases2) # 这里的y既是神经网络的输出，输出之后会通过例如sigmoid之类的处理
    
    global_step = tf.Variable(0, trainable=False) # 训练的轮数，为不可训练的变量
    
    # 滑动平均相关
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 用交叉熵作为损失函数， 直接使用sparse_softmax_cross_entropy_with_logits来进行 softmax回归处理 和 交叉熵的计算
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_,1))
    
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 优化目标中加入L2正则化
    regularizer = tf.contrib.layers.l2_regularizer(REGU_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    Loss = cross_entropy_mean + regularization
    
    # 设置衰减指数学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples/BATCH_SIZE, LEARNING_RATE_DECAY)
    
    # 使用 Gradient Descent 算法来优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(Loss, global_step=global_step)
    
    # 训练滑动平均
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 验证滑动平均模型向前传播是否正确
    correct_prediction = tf.equal(tf.argmax(average_y,1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 开始会话， 开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据， 用于判断神经网络训练过程的停止
        validate_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
        
        # 准备测试数据做模型的评价
        test_feed = {x:mnist.test.images, y_:mnist.test.labels}
        
        # 迭代训练神经网络
        for i in range(TRAINING_STEPS):
            #每1000轮输出一次在验证集上的结果
            if i % 1000 ==0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g" %(i, validate_acc))
                
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs, y_:ys})
            
        # 在训练结束之后，用测试集验证模型准确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training step(s), test accuracy using average model is %g"%(TRAINING_STEPS, test_acc))
        
        

In [52]:
def main(argv=None):
    mnist = input_data.read_data_sets("C:\\Users\\y63qiu\\", one_hot=True)
    train(mnist)

In [53]:
 if __name__ == '__main__':
    tf.app.run()

Extracting C:\Users\y63qiu\train-images-idx3-ubyte.gz
Extracting C:\Users\y63qiu\train-labels-idx1-ubyte.gz
Extracting C:\Users\y63qiu\t10k-images-idx3-ubyte.gz
Extracting C:\Users\y63qiu\t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1386
After 1000 training step(s), validation accuracy using average model is 0.9774
After 2000 training step(s), validation accuracy using average model is 0.9816
After 3000 training step(s), validation accuracy using average model is 0.9828
After 4000 training step(s), validation accuracy using average model is 0.9834
After 5000 training step(s), validation accuracy using average model is 0.984
After 6000 training step(s), validation accuracy using average model is 0.984
After 7000 training step(s), validation accuracy using average model is 0.9848
After 8000 training step(s), validation accuracy using average model is 0.9844
After 9000 training step(s), validation accuracy using average model is 0.985
Af

SystemExit: 

C:\Users\y63qiu\AppData\Local\Continuum\anaconda3\envs\yorkml\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
